# Decorators and Context Managers 
# Python 3.6 (but all this stuff is available in 2.7)
## PyHou Meetup Feb18
---
##### [Ryan Thielke ](https://www.linkedin.com/in/ryan-thielke-b987012a/) 
##### [Two Sigma ](https://www.twosigma.com) 

# Agenda
---
### 1) Decorators
```python
@deco
def my_func():
    pass
```
### 2) Context Managers
```python
with context_man() as cm:
    do_stuff()
```
### 3) Q&A - But feel free to stop me at any time

# Decorators
---
In Python, everything is an object. You are all very familiar with passing strings, ints, or floats to functions. But what about other functions? Decorators are just that, a convenient syntax for functions that accept and return functions. 

You might have seen these in:
* classmethod / staticmethod annotations in class definitions
* unittest skips and expected failures 
* asyncio coroutines
* flask routes


[For our classically trained computer scientists] It's similar but not exactly the GoF [decorator design patern](https://en.wikipedia.org/wiki/Decorator_pattern).

In [ ]:
import time

def timer(function): 
    """
        A decorator that will calculate the execution time of a function call
    """
    def wrapper(*args, **kwargs): 
        start = time.time()
        value = function(*args, **kwargs) 
        end = time.time()
        print("{} took {} secs to execute".format(function.__name__, end - start))
        return value
    return wrapper 
        

In [ ]:
@timer
def my_func(n):
    """
        Sleeps and then returns the input value
    """
    time.sleep(.1 * n)
    return n

my_func(20)

In [ ]:
# The decorator annotation is just nicer syntax for:
def my_func1(n):
    """
        Sleeps and then returns the input value
    """
    time.sleep(.1 * n)
    return n

timer(my_func1)(20)
timer_my_func1 = timer(my_func1)
timer_my_func1(20)

In [ ]:
# One problem...
my_func??

In [ ]:
from functools import wraps #Tools for higher-order functions

def memo(function):
    """
        Cache results of expensive function calls AKA memoize
    """
    cache = {}
    @wraps(function)
    def wrapper(*args):
        if args in cache.keys():
            print("Retrieving value from the cache")
            return cache[args]
        print("Calling {} to compute a new value".format(function.__name__))
        value = function(*args)
        print("Saving in the cache")
        cache[args] = value
        return value
    return wrapper

In [ ]:
@memo
def my_func2(n):
    """
        A function used to sleep for 3 secs and then returns the input plus 10
    """
    time.sleep(3)
    return n + 10

In [ ]:
# Take another look at the function man page
my_func2??

In [ ]:
%time my_func2(5)

In [ ]:
%time my_func2(10)

In [ ]:
%time my_func2(5)

In [ ]:
def retry(max_attempts=5, sleep_time=5):
    """
        This is a decorator that will call an 
        arbitrary function multiple times 
        with a small delay between each call
        
        Arguments:
            max_attempts(int) -> maximum number of retries before throwing an error
            sleep_time  (int) -> number of secs to sleep between each call
    """
    def decorator(function):
        """ This is actually the decorator """
        @wraps(function)
        def wrapper(*args, **kwargs):
            """ This is where the magic happens """
            for attempt in range(max_attempts):
                try:
                    return function(*args, **kwargs)
                except Exception as e:
                    print("[ERROR] Attempt {} of {}: {}".format(attempt + 1, max_attempts, e))
                    if attempt + 1 == max_attempts:
                        raise e
                    print("[ERROR] Retrying again in {} sec(s)".format(sleep_time))
                    time.sleep(sleep_time)
        return wrapper
    return decorator

In [ ]:
#Slightly different syntax! 
@retry()
def failure_func():
    raise
    
failure_func()

In [ ]:
global_var = 1

@retry(max_attempts=3, sleep_time=3)
def one_excep_then_success():
    """
        Raise one exception and then return True
    """
    global global_var
    if global_var % 2 != 0:
        global_var += 1
        raise
    return True

one_excep_then_success()
# decorator                          function                function arguments 
# retry(max_attempts=3, sleep_time=3)(one_excep_then_success)()

In [ ]:
one_excep_then_success??

# Context Managers
---
From [the Python docs](https://docs.python.org/2/reference/datamodel.html#with-statement-context-managers):
>A context manager is an object that defines the runtime context to be established when executing a with statement. The context manager handles the entry into, and the exit from, the desired runtime context for the execution of the block of code. Context managers are normally invoked using the with statement (described in section The with statement), but can also be used by directly invoking their methods.

>Typical uses of context managers include saving and restoring various kinds of global state, locking and unlocking resources, closing opened files, etc.

You can also write context managers with generator expressions, but we will only look at the original protocol.

In [ ]:
with open("my_file.txt", 'w') as f_out:
    f_out.write("Hello, world!\n")
    
# Equivalent to 
# try:
#     f_out = open("my_File.txt", 'w')
#     f_out.write("Hello, world!\n")
#     ...
# finally:
#     f_out.close()

!cat my_file.txt
!rm my_file.txt

In [ ]:
class code_block_timer:
    def __init__(self):
        pass
    
    def __enter__(self):
        self.start = time.time()
        
    def __exit__(self, *excep):
        self.end = time.time()
        print("This code block took {} secs to execute".format(self.end - self.start))

In [ ]:
with code_block_timer():
    my_func(10) # ~ 1 sec
    my_func(10) # ~ 1 sec
    my_func(20) # ~ 2 secs

In [ ]:
with code_block_timer() as cbt:
    failure_func()
    
# if __enter__() executes without error
# __exit__() is guaranteed to be called

In [ ]:
class swallow_exception(object):
    """Will swallow the first exception and the program will continue"""
    def __init__(self):
        pass
    
    def __enter__(self):
        pass
    
    def __exit__(self, e_type, e_val, trcbk):
        """
            e_type (bool) -> if an exception occured during execution
            e_val  (str)  -> value of the exception
            trcbk         -> stack trace of the execption
        """
        if e_type:
            print("Gulp")
            print("That {} was tasty".format(e_val))
        # return a truthy value to suppress the exception
        return True

with swallow_exception():
    failure_func()
    print("I WONT PRINT")
    failure_func()
print("\nLet the program continue!")

In [ ]:
import os
import sqlite3

class start_transaction:
    """
        A transaction context manger for interacting with a sqlite3 db
        db and schema must already be created!
    """
    def __init__(self, db_name):
        self.db_name = db_name
        
    def __enter__(self):
        if not os.path.exists(self.db_name):
            raise RuntimeError("Please provide a created db")
        self.db = sqlite3.connect(self.db_name)
        self.cursor = self.db.cursor()
        print("Starting the transaction")
        return self.cursor # This will let you hook into the class
    
    def __exit__(self, e_type, e_val, trcbk):
        if not e_type:
            print("Looks good, commiting the transaction")
            self.db.commit()
        else:
            print("Whoops, something went wrong: {}".format(e_val), 'Rolling back...', sep='\n')
            self.db.rollback()
        self.db.close()
        return True # Won't reraise the exception
        

In [ ]:
db_name = "my_test.db"

conn = sqlite3.connect(db_name)
conn.execute("create table pyhou_members(id integer primary key, name varchar(50))")
conn.commit()
conn.close()

!ls -la *.db

In [ ]:
data_chunk1 = [(1, 'Ryan T'),
               (2, 'David M'),
               (3, 'Charlie H')]
data_chunk2 = [(3, 'Decisio Health'), (4, 'Two Sigma')]

print("Step1 Inserting data_chunk1")
with start_transaction(db_name) as transaction:
    transaction.executemany("insert into pyhou_members(id, name) values (?,?)", data_chunk1)
    
print("\nStep2 Inserting data_chunk2")
with start_transaction(db_name) as transaction:
    transaction.executemany("insert into pyhou_members(id, name) values (?,?)", data_chunk2)

print("\nStep3 Retrieving data")
with start_transaction(db_name) as transaction:
    members = transaction.execute("select * from pyhou_members")
    print("Current members: ")
    for member in members.fetchall():
        print('\t', member)
        
os.remove(db_name)

In [ ]:
from contextlib import ContextDecorator #So many cool Context Manager tools in this lib
from datetime import datetime

class stdout_redirect(ContextDecorator):
    """
        A context manager used to highjack the regular print function. 
        
        Any print function that is called within this context will write to stdout as well as a log file.
        At the conclusion of the block, the print function is returned back to normal
    """
    def __init__(self, log_filename=None):
        default_name = datetime.now().strftime("%Y%m%d.log")
        self.log_filename = log_filename or default_name
        
    def __enter__(self):
        global print
        self.builtin_print = print
        self.builtin_print("[INFO]: Redirecting stdout to {}".format(self.log_filename))
        
        def redirect(*args, **kwargs):
            """Redirects stdout to a log file"""
            sep = kwargs.get('sep', ' ')
            end = kwargs.get('end', '\n')
            with open(self.log_filename, 'a') as my_log:
                self.builtin_print(*args)
                my_log.write(sep.join(args) + end)
        
        print = redirect
        
    def __exit__(self, *exc):
        global print
        print = self.builtin_print
        print("[INFO]: No longer redirecting stdout")
        

In [ ]:
print("This is not in the log\n")

with stdout_redirect("my_log.txt"):
    print("Informative print statement #1")
    print("Informative print statement #2")
    
print("\nThis should not be in the log\n")

!echo "-----------"
!ls -lh my_log.txt
!echo "-----------"
!cat my_log.txt
!rm my_log.txt

In [ ]:
@stdout_redirect()
def my_greeter(name):
    print("Hello, {}!".format(name))

print("This should not be in the log \n")

my_greeter("Ryan")

print("\nThis should not be in the log\n")

!echo "-----------"
!ls -lh $(date +"%Y%m%d").log
!echo "-----------"
!cat $(date +"%Y%m%d").log
!rm $(date +"%Y%m%d").log

##  Standard lib package docs
[Python 3.6 functools docs](https://docs.python.org/3/library/functools.html)

[Python 3.6 contextlib docs](https://docs.python.org/3/library/contextlib.html)

## Any questions?